In [2]:
from UDRead import *
from parse_asd import *
from get_atomic_term import *

parity = "even"
order = "E"
# nist_max=0

Atom = "Ca I"
ri = True
nist_max = 51 if parity=="odd" else 45 # Ca I

# Atom = "Sr I"
# ri = False
# nist_max = 50 if parity== "odd" else 50 # Sr I

# Atom = "In II"
# ri = False
# nist_max = 36 if parity== "odd" else 32 # Sr I

# Atom = "In I"
# ri = False
# nist_max = 34 if parity== "odd" else 27 # Sr I

# Atom = "Y II"
# ri = False
# nist_max = 43 if parity== "odd" else 40 # Sr I

# Atom = "Cs VII"
# ri = True # fac=2 for Sr, fac=4 for Ca
# nist_max = 32 if parity== "odd" else 31 # Sr I

Name = Atom.replace(" ","_")

# # DATA NIST
# Store Filtered data of even or odd parity in DATA_Filtered/NIST/ 
url_nist = generate_asd_url(Atom,parity)
data_nist = generate_df_from_asd(url_nist)
data_nist = reformat_df_to_atomdb(data_nist)
df_to_csv(data_nist,"DATA_Filtered/NIST/"+Atom,parity,ri=ri)

## DATA UD
# Sorted formatting, Converted .RES to .csv and stored Filtered data of even or odd parity in DATA_Filtered/UD/ 
convert_res_to_csv("DATA_RAW/UD/"+Name+"_UD_"+parity.capitalize()+".RES")

path_nist = "DATA_Filtered/NIST/"+Name+"_NIST_"+parity.capitalize()+".csv"
path_ud = "DATA_Filtered/UD/"+Name+"_UD_"+parity.capitalize()+".csv"

data = MainCode(path_nist, path_ud,nist_max,Ordering=order)
data_final = Missing_Levels(data)


# # Export
path = "DATA_Output/"+Name+"_"+parity.capitalize()+".txt" #specify path for export
ConvertToTXT(data_final,path)

nist_max :  45
Generating Dataframes...!
Correcting in ud configurations
Correcting in ud configurations : Complete..!
Finding Correspondance
Finding Correspondance : Complete...!
Duplicates :  []
Sorted by energies...!
Finding missing levels
Finding missing levels : complete...!
Produced Output File at DATA_Output/Ca_I_Even.txt
Done...!


In [109]:
config_nist,term_nist,J_nist,config_ud,term_ud,J_ud = data_final[:,0],data_final[:,1],data_final[:,2],data_final[:,5],data_final[:,8],data_final[:,9]

idx = np.where(config_nist=="-")[0]
config_nist[idx],term_nist[idx],J_nist[idx]=config_ud[idx],term_ud[idx],J_ud[idx]
terms_all = np.array([term_nist[i]+J_nist[i] for i in range(len(term_nist))])

lst = np.array([])
config_exp,term_exp,J_exp=[],[],[]
for i in range(len(config_nist)):
    config = config_nist[i]
    if (i in lst)==True or ("0" in config)==True: continue
    terms_expected = scrape_term(config)
    for j in range(len(terms_expected)):
        idx = np.where((config_nist==config) & (terms_all==terms_expected[j]))[0]
        lst = np.concatenate([lst,idx])
        if len(idx)==0:
            config_exp.append(config_nist[i])
            term_exp.append(terms_expected[j][0:2])
            J_exp.append(terms_expected[j][-1])
            
data_csv=[]
data_csv.append(["-","-","-","-","-","-","-","-","-","-","-","-","-","-","-","-","-"])
for j in range(len(config_exp)):
    data_csv.append([config_exp[j],term_exp[j],J_exp[j],"-","-","-","-","-","-","-","-","-","-","-","-","-","-"])

array([['4s.4p', '3P', '0', ..., '0%', '95.1%', ''],
       ['4s.4p', '3P', '1', ..., '2.68%', '95.1%', ''],
       ['4s.4p', '3P', '2', ..., '3.48%', '95.0%', ''],
       ...,
       ['4s.7h', '1H', '5', ..., '-', '-', '-'],
       ['4s.9p', '3P', '0', ..., '-', '-', '-'],
       ['4s.8h', '1H', '5', ..., '-', '-', '-']], dtype='<U10')

In [18]:
ManualCorrection=True if Atom == "Sr I" else False
new_config = Corrected_Config(df_ud,df_nist,ManualCorrection)

In [19]:
Ordering=order

data_csv = []
ao_used = []
roundd = 3

ref_E = 0
for i in range(len(df_nist)):
    config_nist, term_nist, j_nist, level_nist, uncer_nist = Data_Nist(i,df_nist)
    data_nist = [config_nist, term_nist, j_nist, round(level_nist+ref_E,roundd),uncer_nist]

    nist_used = 0

    jth,jf,de = FindJth(i,df_nist,df_ud,corr_config=new_config,which_j=True)
#     if jf==2 and (1 in IsBeingUsed(jth,df_nist,df_ud,corr_config=new_config))==True:jth=-1 # avoiding duplicates
    if jth!=-1:
        ao_used.append(jth)
        nist_used = 1
        config1_ao, config2_ao, term_ao, j_ao, level_ao,level_au, per1,per2,uncer_ud = Data_UD(jth,df_ud)
        final_config = config2_ao if jf==2 else config1_ao # final config
        Ediff = round(abs(level_nist-level_ao),1)
        Eperc = round(100*Ediff/level_nist,2) if level_nist!=0 else 0
        EpercStr = str(Eperc)+"%"
        if config2_ao=='': config2_ao='-'
        data_csv.append(data_nist+["-",new_config[jth],config1_ao,config2_ao, term_ao, j_ao, round(level_ao+ref_E,roundd),uncer_ud,level_au,Ediff,EpercStr, per1,per2])


    ## remaining nist states
    if nist_used==0:
        data_csv.append([config_nist, term_nist, j_nist, round(level_nist+ref_E,roundd),uncer_nist,"-","-","-","-","-","-",round(level_nist+ref_E,roundd),"-","-","-","-","","-"])

## remaining ao states
for j in range(len(df_ud)):
    config1_ao, config2_ao, term_ao, j_ao, level_ao,level_au, per1,per2,uncer_ud = Data_UD(j,df_ud)
    if config2_ao=='': config2_ao='-'
    if (j in ao_used)==False:
        data_csv.append(["-","-","-",round(level_ao+ref_E,roundd),"-","-",new_config[j],config1_ao, config2_ao, term_ao, j_ao, round(level_ao+ref_E,roundd),uncer_ud,level_au,"-","-", per1,per2])


header = [" Config","  Term","  J","  Level (cm⁻¹)","Uncertainty (cm-1)","Final Config","   OrgConfig1","  CorConfig1","  Config2","  Term","  J","  Level (cm⁻¹)","  Uncertainty (cm⁻¹)","  Level (au)","    \u0394E","    \u0394E%","  I","  II"]
data = np.array(data_csv)

# sort data
sort = data.T[3].astype(float).argsort()
data = data[sort]

for i in range(len(data)):
    if data[i][0]=="-": data[i][3] = "-"
    if data[i][0]=="": data[i][3] = ""

for i in range(len(data)):# 6 and 10 neends to change if added any new column between them
    if data[i][6]=="-": data[i][11] = "-"
    if data[i][6]=="": data[i][11] = ""

print(np.where(np.bincount(ao_used) > 1)[0]) # duplicates

if Ordering=="E": data_final = FinalSortE(data)
# if Ordering=="T": data_final = FinalSortT(data)

[]


In [11]:
data = [header,data_final]

remove = [-1,-2]
data_new = np.delete(data[1], remove, axis=1)
header_new = np.delete(data[0], remove, axis=0)

In [13]:
pd.DataFrame(data_new,columns=header_new)

# # Export
# path = "DATA_Output/"+Name+"_"+parity.capitalize()+".txt" #specify path for export
# ConvertToTXT(data,path)

,Config,Term,J,Level (cm⁻¹),Uncertainty (cm-1),Final Config,OrgConfig1,CorConfig1,Config2,Term,J,Level (cm⁻¹),Uncertainty (cm⁻¹),Level (au),ΔE,ΔE%
0,4s2,1S,0,0,,-,4s2,4s2,-,1S,0,0.0,,0.66324847,0.0,0%
1,3d.4s,3D,1,20335.36,,-,4s.3d,4s.3d,-,3D,1,20211.4,,0.57115838,124.0,0.61%
2,3d.4s,3D,2,20349.26,,-,4s.3d,4s.3d,-,3D,2,20231.8,,0.57106557,117.5,0.58%
3,3d.4s,3D,3,20371,,-,4s.3d,4s.3d,-,3D,3,20263.0,,0.57092361,108.0,0.53%
4,3d.4s,1D,2,21849.634,,-,4s.3d,4s.3d,-,1D,2,21837.4,,0.56375007,12.2,0.06%
5,4s.5s,3S,1,31539.495,,-,4s.5s,4s.5s,-,3S,1,31720.6,,0.51871881,181.1,0.57%
6,4s.5s,1S,0,33317.264,,-,4s.5s,4s.5s,-,1S,0,33488.4,,0.51066398,171.1,0.51%
7,4s.4d,1D,2,37298.287,,-,4s.4d,4s.4d,4s.5d,1D,2,37450.8,,0.4926102,152.5,0.41%
8,4s.4d,3D,1,37748.197,,-,4s.4d,4s.4d,4s.5d,3D,1,37918.2,,0.49048029,170.0,0.45%
9,4s.4d,3D,2,37751.867,,-,4s.4d,4s.4d,4s.5d,3D,2,37922.7,,0.49045998,170.8,0.45%


In [6]:
# 100*(47242.2-47040.007)/47040.007,100*(47242.2-47466.014)/47466.014

In [24]:
# IsBeingUsed(49,df_nist,df_ud,corr_config=new_config,OutAll=True)
# IsBeingUsed(49,df_nist,df_ud,OutAll=True)

In [54]:
df_ud[:,1][22]

'3d2'

In [12]:
maxx = df_ud[:,1][22]
max(np.where(df_nist[:,0]==maxx)[0])

24

In [13]:
np.intersect1d(df_nist[:,0],df_ud[:,1])

array(['3d 5s', '3d2', '4p2', '4s 4d', '4s 5s', '4s 6d', '4s 6s', '4s 7d',
       '4s 7s', '4s 8d', '4s 8s', '4s 9s', '4s2'], dtype=object)

In [79]:
np.nonzero(np.in1d(df_nist[:,0],df_ud[:,1]))[0]

array([ 0,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 22, 23, 24,
       25, 26, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43,
       44, 47, 48, 49, 50, 77, 78, 79])

In [80]:
np.in1d(df_nist[:,0],df_ud[:,1])

array([ True, False, False, False, False,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
       False, False, False, False,  True,  True,  True,  True,  True,
       False, False,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
       False, False,  True,  True,  True,  True, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False,  True,  True,  True, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False])

In [81]:
df_nist[:,0][79]

'3d2'

In [14]:
new_config = np.full(len(df_ud),"",dtype=object)
list_config,list_term,list_number,count = StartingConfigs(df_nist)

for j in range(len(df_ud)):
    nf,jf,de = IsBeingUsed(j,df_nist,df_ud)
    config1_ao, config2_ao, term_ao, j_ao, level_ao,Levelau, per1,per2,uncer_ud = Data_UD(j,df_ud)
    mul = int(term_ao[0]) if jf==0 else int(df_nist[nf][1][0])
#     print(j,nf,jf)

    if jf==1: config = config1_ao
    if jf==0 or jf==2:
        config = config1_ao
        new_config_check = Correct_Config(j,config,mul,new_config,list_config,list_term,list_number,count,Final=False)
        nf1,jf1,de1 = IsBeingUsed(j,df_nist,df_ud,corr_config=new_config_check)

        config = config2_ao
        new_config_check[j] = config2_ao
        nf2,jf2,de2 = IsBeingUsed(j,df_nist,df_ud,corr_config=new_config_check)
        
        if de1!=0 and de2==0: config=config1_ao
        elif de1==0 and de2!=0: config=config2_ao
        elif de1!=0 and de2!=0: config=config1_ao if de1<=de2 else config2_ao
        else: config=config1_ao
        print(j,config)

    new_config,list_config,list_term,list_number,count = Correct_Config(j,config,mul,new_config,list_config,list_term,list_number,count,Final=True)
#     print(j,config,new_config[j],config1_ao,config2_ao,term_ao,mul)
#     print(j,list_number)
#     print(j,new_config[j],"   ",config1_ao,"   ",config2_ao,"   ",term_ao,"   ",nf1,nf2,mul,de1,de2)
    
#     print(j,IsBeingUsed(j,df_nist,df_ud,corr_config=new_config,OutAll=True))

2 5s2 7p
3 5s2 7p
6 5s2 6f
10 5s2 8f
11 5s2 7f
16 5s2 12p
19 5s2 12p
22 5s2 13p
23 5s2 13p
24 5s2 12f
25 5s2 12f
26 5s2 13p
27 5s2 13p
30 5s2 11f
31 5s2 11f
32 5s 5p 6s
33 5s2 14p
34 5s2 15p
35 5s 5p 6s
36 5s 5p 6s
37 5s 5p 6s
38 5s 5p 6s
39 5s2 15p


In [15]:
new_config

array(['5s2 5p', '5s2 5p', '5s2 6p', '5s2 6p', '5s2 7p', '5s2 7p',
       '5s2 4f', '5s2 4f', '5s2 8p', '5s2 8p', '5s2 5f', '5s2 5f',
       '5s2 9p', '5s2 9p', '5s2 6f', '5s2 6f', '5s2 10p', '5s2 7f',
       '5s2 7f', '5s2 10p', '5s2 8f', '5s2 8f', '5s2 11p', '5s2 11p',
       '5s2 9f', '5s2 9f', '5s2 12p', '5s2 12p', '5s2 10f', '5s2 10f',
       '5s2 11f', '5s2 11f', '5s 5p 6s', '5s2 13p', '5s2 13p', '5s 5p 6s',
       '5s 5p 6s', '5s 5p 6s', '5s 5p 6s', '5s2 14p'], dtype=object)

In [16]:
12,int(abs(9.0-12)-1)

(12, 2)

In [17]:
Sep_Config('5s2 12f')

['5', 's2', '12', 'f']

In [45]:
list_term,"\n",list_number,list_config

(array([2., 2.]),
 '\n',
 array([11.,  9.]),
 array(['5s2 1p', '5s2 1f'], dtype='<U32'))

In [93]:
new_config[49]='4s 8d'

In [96]:
# jth=49
# IsBeingUsed(jth,df_nist,df_ud,corr_config=new_config,OutAll=True)
# IsBeingUsed(jth,df_nist,df_ud,OutAll=True)

In [27]:
# FindJth(41,df_nist,df_ud,corr_config=new_config,which_j=True)
FindJthAll(32,df_nist,df_ud)

([], [], [], [])

In [46]:
jth=37
new_config[jth],df_ud[jth]

('5s 5p 6s',
 array([38, '5s 5p 6s', '4P5/2', 1.62382237, 69369.5, 1.5, 1, 2.5, '1.6',
        '71.1%', '', '', '', ''], dtype=object))

In [78]:
j=42

jf=[]
nf=[]
de=[]
for i in range(len(df_nist)):
    j_final,jw,dE=FindJthAll(i,df_nist,df_ud,corr_config=[],which_j=True)
    if j_final == j:
        jf.append(jw)
        nf.append(i)
        de.append(dE)
        
jf,i

TypeError: FindJthAll() got an unexpected keyword argument 'which_j'

In [31]:
# FindJth(32,df_nist,df_ud,corr_config=[],which_j=True)

In [30]:
df_nist[32]

array(['5s 5p  6s', '4P', 2.5, 68821.7, 2.5, False], dtype=object)

In [65]:
# jth,jf,dE = FindJth(i,df_nist,df_ud,corr_config=new_config,which_j=True)
# print(jth,jf,dE)
# print(new_config[jth])
# IsBeingUsed(jth,df_nist,df_ud,corr_config=new_config,OutAll=True)
# # IsBeingUsed(jth,df_nist,df_ud)

In [7]:
FindJthAll(32,df_nist,df_ud)

5s 5p  6s      5s 5p  6s      5s 5p 6s


([], [], [], [])

In [6]:
def FindJthAll(i,df_nist,df_ud,corr_config=[]): # find ao config corresponding to ith nist config
    j1,j2,Eperc1,Eperc2=[],[],[],[]
    config_nist, term_nist, j_nist, level_nist, uncer_nist = Data_Nist(i,df_nist)
    corr=1 if len(corr_config)==len(df_ud) else 0 # corr=True --> Corrected Configurations are proveded and to use those
    for j in range(len(df_ud)):
        config1_ao, config2_ao, term_ao, j_ao, level_ao,Levelau, per1,per2,uncer_ud = Data_UD(j,df_ud)
        if corr==1: config1_ao=corr_config[j]
            
        Ediff = round(abs(level_nist-level_ao),1)
        Eperc = round(100*Ediff/level_nist,2) if level_nist!=0 else 0
        
        # First Pass : if J_Nist = J_AO
        if j_nist==j_ao:
#             print(j,j_nist,j_ao,type(j_nist),type(j_ao))
            inv_config = Inverse_Config(config_nist)
            
            # Second Pass : if Term_NIST = Term_AO
            if term_nist==term_ao:
                print(config_nist,"    ",inv_config,"    ",config1_ao)
                # Third Pass :
                if config_nist==config1_ao or inv_config==config1_ao:j1.append(j),Eperc1.append(Eperc)
                if corr==0:
                    if config_nist==config2_ao or inv_config==config2_ao:j2.append(j),Eperc2.append(Eperc)
            
            # Fourth Pass : If Term_NIST != Term_AO --> Check using term correction by +-1
            if term_nist!=term_ao: # correction to terms in ao results
                if config_nist==config1_ao or inv_config==config1_ao:
                    if term_nist==Term_Correct(term_ao,1):j1.append(j),Eperc1.append(Eperc)
                    if term_nist==Term_Correct(term_ao,-1):j1.append(j),Eperc1.append(Eperc)

                if corr==0:
                    if config_nist==config2_ao or inv_config==config2_ao:
                        if term_nist==Term_Correct(term_ao,1):j2.append(j),Eperc2.append(Eperc)
                        if term_nist==Term_Correct(term_ao,-1):j2.append(j),Eperc2.append(Eperc)
    
    return j1,j2,Eperc1,Eperc2 # j1,j2 is the index of ud configurations 1 and 2 corresponding to NIST

In [48]:
j=25

config1_ao, config2_ao, term_ao, j_ao, level_ao,Levelau, per1,per2,uncer_ud = Data_UD(j,df_ud)
config1_ao,config2_ao,new_config[j]

('4s 8s', '4s 7s', '4s 7s')

,0,1,2,3,4,5
0,NaN,1S,0,0.0,NaN,False
1,5s 4d,3D,1,18159.04,0.003,False
2,5s 4d,3D,2,18218.784,0.003,False
3,5s 4d,3D,3,18319.261,0.004,False
4,5s 4d,1D,2,20149.685,0.003,False
5,5s 6s,3S,1,29038.773,0.004,False
6,5s 6s,1S,0,30591.825,0.016,False
7,5s 5d,1D,2,34727.447,0.005,False
8,5s 5d,3D,1,35006.908,0.007,False
9,5s 5d,3D,2,35021.989,0.012,False


In [5]:
new_config = np.full(len(df_ud),"",dtype=object)
list_config,list_term,list_number,count = StartingConfigs(df_nist)

In [6]:
conf=Sep_Config(config)
dummy_config = Dummy_Config(config)
idx_dummy = np.where((list_config.astype(str)==dummy_config) & (list_term.astype(int)==mul))[0]

n = int(conf[-2])
# Store term and config for comparison
if len(idx_dummy)==0:
    list_config=np.append(list_config,dummy_config)
    list_term=np.append(list_term,mul)
    list_number=np.append(list_number,n)
    count = np.append(count,0)
    
# main correction part
idx_dummy = np.where((list_config.astype(str)==dummy_config) & (list_term.astype(int)==mul))[0] # index of dummy config present in list_config
if conf[-1]=='s':mul=1
n_prev = list_number[idx_dummy[0]]
if abs(n_prev-n)>1:
    n=n-int(abs(n_prev-n)-1)


conf[-2]=str(n)
new_config[jth]=Combine_Config(conf)


if count[idx_dummy[0]]!=mul: count[idx_dummy[0]]+=1
# re-initian for the next loop
if count[idx_dummy[0]]==mul:
    list_number[idx_dummy[0]]=n
    count[idx_dummy[0]]=0
    
mul,count,new_config,list_number,list_config

(2,
 array([0., 1., 0.]),
 array(['', '', '', '', '5s2 5d', '', '', '', '', '', '', '', '', '', '',
        '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
        '', '', '', '', '', '', '', ''], dtype=object),
 array([5., 4., 4.]),
 array(['1s2 1s', '1s2 1d', '1s 1p2'], dtype='<U32'))

In [22]:
jth,jf,dE = FindJth(i,df_nist,df_ud,corr_config=new_config,which_j=True)
print(jth,jf,dE)
print(new_config[jth])
# IsBeingUsed(jth,df_nist,df_ud,corr_config=new_config,OutAll=True)
IsBeingUsed(jth,df_nist,df_ud)

4 1 37.81
5s2 5d


(7, 1, 20.0)

In [10]:
j=6

config1_ao, config2_ao, term_ao, j_ao, level_ao,Levelau, per1,per2,uncer_ud = Data_UD(j,df_ud)
config1_ao, config2_ao, term_ao, j_ao

('5s2 8s', '5s2 7s', '2S', 0.5)

In [12]:
# StartingConfigs(df_nist)

In [6]:
i=10
FindJthAll(i,df_nist,df_ud)

([8], [], [2.37], [])

In [36]:
# df_nist[i]

In [35]:
# OrgConfig1,config1_ao, config2_ao, term_ao, j_ao, level_ao,Levelau, per1,per2,uncer_ud = Data_UD(i,df_ud)
# OrgConfig1,config1_ao, config2_ao, term_ao, j_ao

''